In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [72]:
df = pd.read_csv('final_df2.csv',index_col=0)

In [73]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [74]:
from sklearn.preprocessing import StandardScaler

std_df = StandardScaler().fit_transform(X)
std_df = pd.DataFrame(std_df, index=X.index, columns=X.columns)
df2 = std_df
df2.head()

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,0.886827,0.349044,0.159261,-1.586876,-0.783517,-0.651792,-0.192552,-0.688943,1.790315,0.606942,...,0.618291,-0.244858,0.087421,-0.441970,0.805609,0.084007,-0.853409,-0.767943,-0.097567,0.208945
1,-0.076357,0.902533,0.026440,1.624105,1.396133,0.195322,0.394281,0.733565,1.460883,-0.288978,...,0.083736,-0.957388,0.095699,0.972672,0.691112,1.004964,-0.178898,-0.945141,0.843130,-1.062326
2,1.160544,0.875732,4.027129,0.072768,0.488563,-0.707819,-1.258655,2.353938,1.514641,-0.194159,...,0.561794,-0.680456,-0.348212,0.766775,-0.167253,0.878351,1.027238,-0.487267,0.630930,-0.273803
3,-0.093755,0.023215,-1.310286,0.819120,-0.815949,-0.407544,0.793497,-0.158602,-0.161729,1.060372,...,2.103530,-1.246807,-1.676863,0.213831,-1.585708,0.933448,0.740015,-1.050423,-0.987469,-0.644158
4,-0.249586,-0.633865,-0.325074,0.155425,0.019339,-0.462644,-0.576603,-1.569976,-1.943742,0.022223,...,-1.478282,0.569776,-0.794047,-0.079013,0.559841,-0.793164,-1.567624,0.733505,-0.957059,0.428773


In [76]:
from sklearn.decomposition import PCA

pca = PCA(n_components=533)
pca_array = pca.fit_transform(X)


In [95]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
# 주성분의 개수를 지정하지 않고 찾아줌
# 분산의 비율을 누적해서 더한 값이 95% 이상인 주성분의 개수
# 즉, 분산의 95%이상 유지
d = np.argmax(cumsum >= 0.99) + 1

In [96]:
d

533

In [75]:
dec=PCA(0.99)
x2=dec.fit_transform(X)
x2.shape

(590, 533)

In [77]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test=train_test_split(pca_array,y,test_size= 0.2, random_state=42)

In [79]:
def estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=accuracy_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [81]:
estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 2.304534912109375 seconds
number 1 out of 300 took 1.6421759128570557 seconds
number 2 out of 300 took 1.646596908569336 seconds
number 3 out of 300 took 1.4640843868255615 seconds
number 4 out of 300 took 1.5129544734954834 seconds
number 5 out of 300 took 1.4999897480010986 seconds
number 6 out of 300 took 1.5977270603179932 seconds
number 7 out of 300 took 1.5299086570739746 seconds
number 8 out of 300 took 1.6815032958984375 seconds
number 9 out of 300 took 1.5867552757263184 seconds
number 10 out of 300 took 1.7194020748138428 seconds
number 11 out of 300 took 1.5638184547424316 seconds
number 12 out of 300 took 1.6116917133331299 seconds
number 13 out of 300 took 1.466078758239746 seconds
number 14 out of 300 took 1.3852949142456055 seconds
number 15 out of 300 took 1.5947363376617432 seconds
number 16 out of 300 took 1.4920108318328857 seconds
number 17 out of 300 took 1.6525797843933105 seconds
number 18 out of 300 took 1.5867574214935303 seconds
number

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 27 out of 300 took 1.3952686786651611 seconds
number 28 out of 300 took 1.691476583480835 seconds
number 29 out of 300 took 1.7403464317321777 seconds
number 30 out of 300 took 1.5727941989898682 seconds
number 31 out of 300 took 1.4660813808441162 seconds
number 32 out of 300 took 1.4271819591522217 seconds
number 33 out of 300 took 1.6844956874847412 seconds
number 34 out of 300 took 1.573791265487671 seconds
number 35 out of 300 took 1.6785120964050293 seconds
number 36 out of 300 took 1.4471302032470703 seconds
number 37 out of 300 took 1.4391522407531738 seconds
number 38 out of 300 took 1.505971908569336 seconds
number 39 out of 300 took 1.629643440246582 seconds
number 40 out of 300 took 1.7034435272216797 seconds
number 41 out of 300 took 1.668539047241211 seconds
number 42 out of 300 took 1.5973186492919922 seconds
number 43 out of 300 took 1.5329012870788574 seconds
number 44 out of 300 took 1.6436059474945068 seconds
number 45 out of 300 took 1.7144157886505127 second

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 114 out of 300 took 1.3424110412597656 seconds
number 115 out of 300 took 1.435161828994751 seconds
number 116 out of 300 took 1.3493916988372803 seconds
number 117 out of 300 took 1.353381872177124 seconds
number 118 out of 300 took 1.441145658493042 seconds
number 119 out of 300 took 1.358367919921875 seconds
number 120 out of 300 took 1.4281806945800781 seconds
number 121 out of 300 took 1.3573706150054932 seconds
number 122 out of 300 took 1.3733277320861816 seconds
number 123 out of 300 took 1.3872900009155273 seconds
number 124 out of 300 took 1.451120376586914 seconds
number 125 out of 300 took 1.4241907596588135 seconds
number 126 out of 300 took 1.350389003753662 seconds
number 127 out of 300 took 1.4032485485076904 seconds
number 128 out of 300 took 1.3892848491668701 seconds
number 129 out of 300 took 1.384298324584961 seconds
number 130 out of 300 took 1.4142193794250488 seconds
number 131 out of 300 took 1.3643507957458496 seconds
number 132 out of 300 took 1.608698

...

number 205 out of 300 took 1.4351630210876465 seconds
number 206 out of 300 took 1.3852951526641846 seconds
number 207 out of 300 took 1.3364267349243164 seconds
number 208 out of 300 took 1.4162120819091797 seconds
number 209 out of 300 took 1.358368158340454 seconds
number 210 out of 300 took 1.4600956439971924 seconds
number 211 out of 300 took 1.344404935836792 seconds
number 212 out of 300 took 1.352384328842163 seconds
number 213 out of 300 took 1.3862922191619873 seconds
number 214 out of 300 took 1.358368158340454 seconds
number 215 out of 300 took 1.3852958679199219 seconds
number 216 out of 300 took 1.3583669662475586 seconds
number 217 out of 300 took 1.3743250370025635 seconds
number 218 out of 300 took 1.4032483100891113 seconds
number 219 out of 300 took 1.369337797164917 seconds
number 220 out of 300 took 1.4172098636627197 seconds
number 221 out of 300 took 1.3533809185028076 seconds
number 222 out of 300 took 1.400256872177124 seconds
number 223 out of 300 took 1.38928

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 235 out of 300 took 1.4062399864196777 seconds
number 236 out of 300 took 1.3573694229125977 seconds
number 237 out of 300 took 1.4221971035003662 seconds
number 238 out of 300 took 1.3633549213409424 seconds
number 239 out of 300 took 1.3493916988372803 seconds
number 240 out of 300 took 1.408233642578125 seconds
number 241 out of 300 took 1.3823037147521973 seconds
number 242 out of 300 took 1.4321706295013428 seconds
number 243 out of 300 took 1.3643519878387451 seconds
number 244 out of 300 took 1.364351511001587 seconds
number 245 out of 300 took 1.4022505283355713 seconds
number 246 out of 300 took 1.3723299503326416 seconds
number 247 out of 300 took 1.4401485919952393 seconds
number 248 out of 300 took 1.3394184112548828 seconds
number 249 out of 300 took 1.3872904777526855 seconds
number 250 out of 300 took 1.4700689315795898 seconds
number 251 out of 300 took 1.3673439025878906 seconds
number 252 out of 300 took 1.4391515254974365 seconds
number 253 out of 300 took 1.3

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 284 out of 300 took 1.4401490688323975 seconds
number 285 out of 300 took 1.4950025081634521 seconds
number 286 out of 300 took 1.4940056800842285 seconds
number 287 out of 300 took 1.349390983581543 seconds
number 288 out of 300 took 1.3274502754211426 seconds
number 289 out of 300 took 1.3932738304138184 seconds
number 290 out of 300 took 1.3174774646759033 seconds
number 291 out of 300 took 1.4062395095825195 seconds
number 292 out of 300 took 1.3523836135864258 seconds
number 293 out of 300 took 1.3304426670074463 seconds
number 294 out of 300 took 1.410229206085205 seconds
number 295 out of 300 took 1.3394184112548828 seconds
number 296 out of 300 took 1.4062390327453613 seconds
number 297 out of 300 took 1.3234617710113525 seconds
number 298 out of 300 took 1.348393440246582 seconds
number 299 out of 300 took 1.4261860847473145 seconds


In [82]:
training_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.0,300.0,300.0
mean,0.999991,1.0,1.0,1.0
std,0.000163,0.0,0.0,0.0
min,0.997175,1.0,1.0,1.0
25%,1.000000,1.0,1.0,1.0
50%,1.000000,1.0,1.0,1.0
75%,1.000000,1.0,1.0,1.0
max,1.000000,1.0,1.0,1.0


In [83]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.924237,0.985763,0.993644,0.997006
std,0.026668,0.011318,0.006795,0.005106
min,0.830508,0.932203,0.966102,0.974576
25%,0.906780,0.983051,0.991525,0.991525
50%,0.923729,0.991525,0.991525,1.000000
75%,0.940678,0.991525,1.000000,1.000000
max,0.983051,1.000000,1.000000,1.000000


In [84]:
timing_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.387489,0.980239,0.027853,0.031850
std,0.040234,0.068334,0.004586,0.012795
min,0.339093,0.911562,0.022918,0.024905
25%,0.369012,0.931247,0.025931,0.027898
50%,0.378017,0.950430,0.026929,0.029888
75%,0.395969,0.998337,0.028895,0.031915
max,0.867039,1.200780,0.090402,0.229084


In [92]:
testing_score_df

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
0,0.915254,0.991525,0.991525,1.000000
1,0.957627,0.991525,0.991525,1.000000
2,0.932203,0.983051,1.000000,1.000000
3,0.923729,0.966102,0.991525,1.000000
4,0.898305,0.983051,1.000000,1.000000
...,...,...,...,...
295,0.949153,0.991525,0.991525,1.000000
296,0.940678,0.966102,0.974576,0.974576
297,0.872881,0.974576,0.991525,0.991525
298,0.915254,0.991525,0.991525,1.000000


In [89]:
testing_score_df.sort_values(by='SVC',ascending=False)

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
150,0.915254,0.983051,1.000000,1.000000
130,0.940678,1.000000,1.000000,1.000000
229,0.940678,0.991525,1.000000,1.000000
106,0.923729,1.000000,1.000000,1.000000
107,0.940678,0.991525,1.000000,1.000000
...,...,...,...,...
296,0.940678,0.966102,0.974576,0.974576
184,0.949153,0.974576,0.974576,0.974576
83,0.906780,0.974576,0.974576,0.974576
18,0.881356,0.966102,0.966102,0.991525


In [91]:
testing_score_df.sort_values(by='RandomForestClassifier',ascending=False)

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
165,0.983051,0.991525,0.991525,0.991525
157,0.974576,0.991525,1.000000,1.000000
222,0.974576,0.983051,0.991525,0.991525
195,0.974576,1.000000,0.991525,0.991525
69,0.974576,1.000000,1.000000,1.000000
...,...,...,...,...
26,0.864407,0.991525,0.991525,1.000000
37,0.855932,0.983051,0.991525,1.000000
207,0.847458,0.983051,0.991525,1.000000
180,0.838983,0.949153,0.991525,0.991525


In [86]:
prediction=SVC(random_state=150)

In [87]:
prediction.fit(x,y)

SVC(random_state=150)

In [88]:
prediction.score(x_test,y_test)

0.9915254237288136

In [65]:
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)

0.993685152999509

In [33]:
df2.shape[1]

17815

In [46]:
from sklearn.decomposition import PCA

pca = PCA(n_components=590)
pca_array = pca.fit_transform(df2)
pca_df = pd.DataFrame(pca_array, index=df2.index,
                      columns=[f"pca{num+1}" for num in range(590)])
pca_df.head()

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca581,pca582,pca583,pca584,pca585,pca586,pca587,pca588,pca589,pca590
0,-33.804476,-2.787982,-5.770776,-30.958220,5.887185,13.407046,2.691613,-14.574579,-9.552975,35.365711,...,0.196188,0.154316,0.327665,0.057420,0.454254,0.043743,-0.494110,0.163552,0.313049,2.829963e-15
1,12.129357,-20.493103,13.751279,-29.977843,-8.738578,-3.651478,-16.750653,18.264358,0.222539,-23.017786,...,-1.231051,0.436648,0.278043,0.841787,0.056323,0.110008,0.127392,0.172217,-0.086814,2.829963e-15
2,-4.647118,-27.232655,-10.584373,-4.731128,-26.303871,-22.174773,5.284348,1.192801,23.295138,2.967032,...,-0.016966,0.286153,0.738451,-0.027324,0.026627,-0.631649,0.330128,-0.128202,0.091551,2.829963e-15
3,-7.638704,-57.561760,7.498658,5.308718,0.283737,-2.635175,-8.665971,1.103797,-17.317805,-5.960301,...,0.261537,-0.382600,0.365680,0.041219,0.090431,-0.025359,0.661129,0.442092,-0.349987,2.829963e-15
4,-31.522639,76.837420,-26.187440,-6.820377,12.824112,-1.214566,11.631132,-9.568467,-10.434659,-4.633602,...,-0.150812,0.078911,0.046604,-0.195311,0.079283,-0.053218,0.263967,0.386297,-0.499102,2.829963e-15


In [49]:
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(590)]))
result['누적기여율'] = result['기여율'].cumsum()
result

,설명가능한 분산 비율(고윳값),기여율,누적기여율
pca1,1.553460e+03,8.705178e-02,0.087052
pca2,1.347472e+03,7.550873e-02,0.162561
pca3,7.642334e+02,4.282560e-02,0.205386
pca4,6.858864e+02,3.843524e-02,0.243821
pca5,4.964501e+02,2.781974e-02,0.271641
...,...,...,...
pca586,2.475582e+00,1.387250e-04,0.999615
pca587,2.370171e+00,1.328181e-04,0.999748
pca588,2.318629e+00,1.299298e-04,0.999878
pca589,2.185088e+00,1.224465e-04,1.000000


In [51]:
# MNIST 데이터 사용하기
from sklearn.datasets import fetch_openml

# 784개의 픽셀(특성)을 가지고 있는 데이터 셋
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)